# Análise de Absentismo - Call Center

**Objetivo:** Análise descritiva e prescritiva de padrões de absentismo.

**Dataset:** 1.3M registos, 3,135 colaboradores, 18 meses (2024-01 a 2025-06)

---

## GRUPO 1: PREPARAÇÃO E LIMPEZA DE DADOS

In [ ]:
# Imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print('Bibliotecas carregadas')

In [ ]:
# 1.1 Carregar dados
print('PASSO 1.1: Carregar dados')
print('-' * 70)

# Dataset principal
df_raw = pd.read_csv('combined_data.csv')
df_raw['Data'] = pd.to_datetime(df_raw['Data'])

print(f'Dataset carregado:')
print(f'  Registos: {len(df_raw):,}')
print(f'  Colaboradores: {df_raw["login_colaborador"].nunique():,}')
print(f'  Periodo: {df_raw["Data"].min().date()} ate {df_raw["Data"].max().date()}')

# Nova classificação
df_codigos = pd.read_excel('códigos_V2.xlsx')

print(f'\nClassificacao carregada:')
print(f'  Codigos: {len(df_codigos)}')
print(f'  Nivel 1 (categorias): {df_codigos["Nivel 1"].nunique()}')
print(f'  Nivel 2 (subcategorias): {df_codigos["Nivel 2"].nunique()}')

In [ ]:
# 1.2 Aplicar classificação
print('\nPASSO 1.2: Aplicar classificacao')
print('-' * 70)

df = df_raw.merge(
    df_codigos,
    left_on='segmento_processado_codigo',
    right_on='Codigo Segmento',
    how='left'
)

# Verificar mapeamento
sem_classificacao = df['Nivel 1'].isna().sum()

if sem_classificacao == 0:
    print('OK: Todos os codigos mapeados')
else:
    print(f'AVISO: {sem_classificacao} registos sem classificacao')

print(f'\nTotal registos: {len(df):,}')

In [ ]:
# 1.3 Identificar incompatibilidades
print('\nPASSO 1.3: Identificar incompatibilidades')
print('-' * 70)

# Matriz de compatibilidade (Nivel 2)
compat_rules = pd.DataFrame([
    # Compativeis
    ['Presença', 'Atraso', 1],
    ['Presença', 'Formação', 1],
    
    # Incompativeis
    ['Presença', 'Ausência Médica', 0],
    ['Presença', 'Ausência Injustificada', 0],
    ['Presença', 'Licença Mat/Pat', 0],
    ['Presença', 'Férias', 0],
    ['Presença', 'Falta Justificada', 0],
    ['Presença', 'Ferias / Feriado / Folga', 0],
    ['Trabalho Pago', 'Ausência Médica', 0],
    ['Trabalho Pago', 'Falta Injustificada', 0],
], columns=['cat1', 'cat2', 'compativel'])

# Criar dicionário
compat_dict = {}
for _, row in compat_rules.iterrows():
    key = tuple(sorted([row['cat1'], row['cat2']]))
    compat_dict[key] = row['compativel']

print(f'Regras de compatibilidade: {len(compat_dict)}')

# Identificar dias com multiplos registos
dias_mult = df.groupby(['login_colaborador', 'Data']).size()
dias_mult = dias_mult[dias_mult > 1].reset_index()
dias_mult.columns = ['login_colaborador', 'Data', 'num_registos']

print(f'\nDias com multiplos registos: {len(dias_mult):,}')

# Pre-filtrar e agrupar (otimizado)
dias_mult_idx = list(zip(dias_mult['login_colaborador'], dias_mult['Data']))
df_mult = df[df.set_index(['login_colaborador', 'Data']).index.isin(dias_mult_idx)]

df_mult_grouped = df_mult.groupby(['login_colaborador', 'Data']).agg({
    'Nivel 2': lambda x: list(x.dropna().unique()),
    'segmento_processado_codigo': lambda x: list(x.unique()),
    'nome_colaborador': 'first'
}).reset_index()

print(f'Dias unicos a testar: {len(df_mult_grouped):,}')

# Testar pares
print('\nTestando incompatibilidades...')
incompativeis = []

for idx, row in df_mult_grouped.iterrows():
    if idx % 10000 == 0 and idx > 0:
        print(f'  Processados {idx:,}/{len(df_mult_grouped):,}')
    
    categorias = row['Nivel 2']
    pares_incompat = []
    
    for i, cat1 in enumerate(categorias):
        for cat2 in categorias[i+1:]:
            key = tuple(sorted([cat1, cat2]))
            if key in compat_dict and compat_dict[key] == 0:
                pares_incompat.append(f'{cat1} + {cat2}')
    
    if pares_incompat:
        incompativeis.append({
            'login_colaborador': row['login_colaborador'],
            'Data': row['Data'],
            'nome_colaborador': row['nome_colaborador'],
            'categorias': ', '.join(categorias),
            'pares_incompativeis': ' | '.join(pares_incompat)
        })

df_incompativeis = pd.DataFrame(incompativeis)

print(f'\nRESULTADO: {len(df_incompativeis)} dias incompativeis encontrados')

if len(df_incompativeis) > 0:
    print('\nDistribuicao por tipo de incompatibilidade:')
    
    # Contar cada par
    todos_pares = []
    for pares_str in df_incompativeis['pares_incompativeis']:
        todos_pares.extend(pares_str.split(' | '))
    
    pares_count = pd.Series(todos_pares).value_counts()
    
    for par, count in pares_count.items():
        print(f'  {par:50s}: {count:3d} casos')
    
    # Exportar
    df_incompativeis.to_excel('incompatibilidades_detalhadas.xlsx', index=False)
    print('\nExportado: incompatibilidades_detalhadas.xlsx')

In [ ]:
# 1.4 Remover dias incompativeis
print('\nPASSO 1.4: Remover dias incompativeis')
print('-' * 70)

if len(df_incompativeis) > 0:
    # Criar indice para remover
    idx_remover = df.set_index(['login_colaborador', 'Data']).index.isin(
        list(zip(df_incompativeis['login_colaborador'], df_incompativeis['Data']))
    )
    
    registos_removidos = idx_remover.sum()
    
    df_limpo = df[~idx_remover].copy()
    
    print(f'Registos removidos: {registos_removidos:,}')
    print(f'Dataset limpo: {len(df_limpo):,} registos')
else:
    df_limpo = df.copy()
    print('Nenhum registo a remover')
    print(f'Dataset limpo: {len(df_limpo):,} registos')

In [ ]:
# 1.5 Separar dataframes
print('\nPASSO 1.5: Separar dataframes')
print('-' * 70)

# df_atrasos: todos os registos onde Nivel 1 = 'Atraso'
# Objetivo: contar numero de DIAS com atraso (nao precisa agregar)
df_atrasos = df_limpo[df_limpo['Nivel 1'] == 'Atraso'].copy()

print(f'df_atrasos criado:')
print(f'  Registos: {len(df_atrasos):,}')
print(f'  Dias unicos: {df_atrasos.groupby(["login_colaborador", "Data"]).ngroups:,}')
print(f'  Colaboradores: {df_atrasos["login_colaborador"].nunique():,}')

# df_absentismo: todos os registos EXCETO atrasos
# Vai ser agregado com hierarquias
df_absentismo = df_limpo[df_limpo['Nivel 1'] != 'Atraso'].copy()

print(f'\ndf_absentismo (antes agregacao):')
print(f'  Registos: {len(df_absentismo):,}')
print(f'  Colaboradores: {df_absentismo["login_colaborador"].nunique():,}')

In [ ]:
# 1.6 Agregar df_absentismo com hierarquias
print('\nPASSO 1.6: Agregar df_absentismo')
print('-' * 70)

# Verificar se ainda ha dias duplicados
dias_dup_abs = df_absentismo.groupby(['login_colaborador', 'Data']).size()
dias_dup_abs = dias_dup_abs[dias_dup_abs > 1]

print(f'Dias com multiplos registos: {len(dias_dup_abs):,}')

if len(dias_dup_abs) > 0:
    # Ver combinacoes que existem
    print('\nCombinacoes existentes (Nivel 1):')
    
    dias_dup_idx = dias_dup_abs.index.tolist()
    df_dup_abs = df_absentismo.set_index(['login_colaborador', 'Data']).loc[dias_dup_idx].reset_index()
    
    combos = df_dup_abs.groupby(['login_colaborador', 'Data'])['Nivel 1'].apply(
        lambda x: ' + '.join(sorted(x.unique()))
    ).value_counts()
    
    for combo, count in combos.items():
        print(f'  {combo:50s}: {count:,} dias')

# Regras de agregacao
# Hierarquia: manter codigo mais relevante para absentismo
# (strings ordenadas para consistencia)
agg_rules = {
    'nome_colaborador': 'first',
    'categoria_profissional': 'first',
    'segmento_processado_codigo': lambda x: ', '.join(sorted(x.unique())),
    'Nivel 1': lambda x: ', '.join(sorted(x.unique())),
    'Nivel 2': lambda x: ', '.join(sorted(x.unique())),
}

# Campos opcionais
for col in ['operacao', 'Activo?', 'DtActivacao', 'DtDesactivacao']:
    if col in df_absentismo.columns:
        agg_rules[col] = 'first'

# Agregar
df_absentismo = df_absentismo.groupby(['login_colaborador', 'Data']).agg(agg_rules).reset_index()

print(f'\ndf_absentismo (apos agregacao):')
print(f'  Dias-colaborador: {len(df_absentismo):,}')
print(f'  Colaboradores: {df_absentismo["login_colaborador"].nunique():,}')
print(f'  Periodo: {df_absentismo["Data"].min().date()} ate {df_absentismo["Data"].max().date()}')

In [ ]:
# 1.7 Validacao final
print('\nPASSO 1.7: Validacao final')
print('=' * 70)

# Check 1: df_absentismo sem duplicados
dup_abs = df_absentismo.groupby(['login_colaborador', 'Data']).size()
dup_abs = dup_abs[dup_abs > 1]

print('\n1. Dias duplicados em df_absentismo:')
if len(dup_abs) == 0:
    print('   OK: Nenhum dia duplicado')
else:
    print(f'   ERRO: {len(dup_abs)} dias duplicados')

# Check 2: df_absentismo sem atrasos
tem_atraso = df_absentismo[df_absentismo['Nivel 1'].str.contains('Atraso', na=False)]

print('\n2. Atrasos em df_absentismo:')
if len(tem_atraso) == 0:
    print('   OK: Nenhum atraso')
else:
    print(f'   ERRO: {len(tem_atraso)} registos com atraso')

# Check 3: df_atrasos so tem atrasos
sem_atraso = df_atrasos[df_atrasos['Nivel 1'] != 'Atraso']

print('\n3. Registos sem atraso em df_atrasos:')
if len(sem_atraso) == 0:
    print('   OK: Todos os registos sao atrasos')
else:
    print(f'   ERRO: {len(sem_atraso)} registos sem atraso')

# Resumo
print('\n' + '=' * 70)
print('RESUMO GRUPO 1:')
print('=' * 70)
print(f'\nDataset original     : {len(df_raw):,} registos')
print(f'Incompatibilidades   : {len(df_incompativeis)} dias removidos')
print(f'\ndf_absentismo        : {len(df_absentismo):,} dias-colaborador')
print(f'                       ({df_absentismo["login_colaborador"].nunique():,} colaboradores)')
print(f'\ndf_atrasos           : {len(df_atrasos):,} registos')
print(f'                       ({df_atrasos.groupby(["login_colaborador", "Data"]).ngroups:,} dias unicos)')
print(f'                       ({df_atrasos["login_colaborador"].nunique():,} colaboradores)')
print('\n' + '=' * 70)
print('Grupo 1 concluido - Dados prontos para analise')
print('=' * 70)